# 도구 호출 에이전트 (Tool Calling Agent)

## 개요

**도구 호출 에이전트** 는 AI 모델이 필요에 따라 외부 도구를 자동으로 선택하고 실행할 수 있도록 해주는 시스템입니다. 이는 AI가 텍스트 생성을 넘어서 실제 업무를 수행할 수 있게 해주는 핵심 기능입니다.

## 도구 호출의 개념

도구 호출(Tool Calling)을 사용하면 AI 모델이 특정 **도구(tool)** 가 **호출되어야 하는 시점을 판단** 하고, 해당 도구에 **적절한 입력값을 전달** 할 수 있습니다.

## 기존 방식과 도구 호출 방식 비교

### 기존 방식의 한계
```
사용자: "오늘 서울 날씨 어때요?"
AI: "죄송합니다. 실시간 날씨 정보에 접근할 수 없습니다."
```

### 도구 호출 방식의 장점
```
사용자: "오늘 서울 날씨 어때요?"
AI → [날씨 API 호출] → "서울은 현재 20도, 맑음입니다."
```

## 실무 활용 비유

**도구 호출 에이전트** 는 회사의 **프로젝트 매니저** 와 유사합니다:

| 역할 | 프로젝트 매니저 | 도구 호출 에이전트 |
|------|-----------------|-------------------|
| 요청 분석 | 고객 요청사항 파악 | 사용자 질문 분석 |
| 팀 배정 | 적절한 전문팀 선택 | 필요한 도구 선택 |
| 업무 지시 | 구체적 작업 내용 전달 | 도구에 파라미터 전달 |
| 결과 취합 | 각 팀 결과 종합 보고 | 도구 결과를 통합하여 답변 생성 |

## 주요 특징

### 지원 모델
- **OpenAI**: GPT-4, GPT-3.5-turbo
- **Anthropic**: Claude 시리즈
- **Google**: Gemini 시리즈
- **Mistral**: Mistral Large 등

### 핵심 장점
- **정확한 도구 호출**: 일반 텍스트 파싱보다 안정적
- **반복 실행 지원**: 문제 해결까지 여러 도구 순차 실행
- **구조화된 출력**: JSON 형태의 명확한 결과
- **오류 처리**: 실패 시 자동 재시도 또는 대안 제시

## 에이전트 실행 과정

```
사용자 질문 → AI 분석 → 도구 선택 → 도구 실행 → 결과 처리 → 최종 답변
```

1. **질문 분석**: 사용자 요청의 의도 파악
2. **도구 선택**: 적절한 도구 식별 및 선택
3. **도구 실행**: 선택된 도구에 필요한 파라미터 전달하여 실행
4. **결과 처리**: 도구 실행 결과를 분석하고 가공
5. **답변 생성**: 처리된 결과를 바탕으로 사용자에게 적절한 답변 제공

## 참고 자료

- [LangChain 도구 호출 에이전트 공식 문서](https://python.langchain.com/v0.1/docs/modules/agents/agent_types/tool_calling/)

![](./assets/agent-concept.png)

In [ ]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv(override=True)

In [ ]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("LangChain-Tutorial")

In [ ]:
from langchain.tools import tool
from typing import List, Dict, Annotated
from langchain_teddynote.tools import GoogleNews
from langchain_experimental.utilities import PythonREPL


# 뉴스 검색 도구 생성
@tool
def search_news(query: str) -> List[Dict[str, str]]:
    """Search Google News by input keyword"""
    # GoogleNews 인스턴스 생성
    news_tool = GoogleNews()
    # 키워드로 뉴스 검색 (최대 5개)
    return news_tool.search_by_keyword(query, k=5)


# Python 코드 실행 도구 생성
@tool
def python_repl_tool(
    code: Annotated[str, "The python code to execute to generate your chart."],
):
    """Use this to execute python code. If you want to see the output of a value,
    you should print it out with `print(...)`. This is visible to the user."""
    result = ""
    try:
        # PythonREPL로 코드 실행
        result = PythonREPL().run(code)
    except BaseException as e:
        # 오류 발생 시 에러 메시지 출력
        print(f"Failed to execute. Error: {repr(e)}")
    finally:
        # 실행 결과 반환
        return result


# 도구 정보 출력
print(f"도구 이름: {search_news.name}")
print(f"도구 설명: {search_news.description}")
print(f"도구 이름: {python_repl_tool.name}")
print(f"도구 설명: {python_repl_tool.description}")

In [ ]:
# 도구들을 리스트로 정의 (에이전트가 사용할 도구 목록)
tools = [search_news, python_repl_tool]

## Agent 프롬프트 생성

**프롬프트** 는 AI 에이전트에게 주는 **업무 지시서** 입니다. 신입사원에게 업무 매뉴얼을 제공하듯이, AI가 어떤 역할을 하고 어떤 도구를 사용해야 하는지 명확히 정의합니다.

## 프롬프트의 핵심 변수

### 주요 구성 요소

| 변수명 | 역할 | 설명 | 활용 시점 |
|--------|------|------|-----------|
| `chat_history` | 대화 기록 저장 | 이전 대화 내용 보존 | 멀티턴 대화 지원 시 |
| `agent_scratchpad` | 임시 작업 메모장 | 도구 호출 과정 및 중간 결과 기록 | 도구 실행 과정에서 |
| `input` | 현재 사용자 요청 | 지금 처리해야 할 질문이나 요청 | 매 요청마다 |

### 변수별 상세 설명

#### 1. chat_history - 대화 기록
- **목적**: 이전 대화 내용을 기반으로 맥락있는 응답 제공
- **데이터 형태**: 이전 사용자 메시지와 AI 응답의 배열
- **주의사항**: 멀티턴 대화가 필요없다면 생략 가능

#### 2. agent_scratchpad - 작업 메모장
- **목적**: AI가 현재 수행 중인 작업의 중간 과정 저장
- **포함 내용**: 도구 호출 시도, 실행 결과, 오류 메시지 등
- **활용**: 복잡한 작업에서 단계별 추론 과정 추적

#### 3. input - 사용자 입력
- **목적**: 현재 처리해야 할 사용자의 직접적인 요청
- **특징**: 매번 새롭게 입력되는 메시지
- **중요성**: 에이전트가 응답해야 할 핵심 질문

## 프롬프트 구조 설계

### 기본 구조
```
시스템 메시지 (업무 매뉴얼)
├── 에이전트 역할 정의
├── 사용 가능한 도구 설명
└── 작업 수행 방식 안내

대화 이력 (이전 대화들)

현재 입력 (처리할 요청)

작업 메모 (진행 중인 내용들)
```

### 효과적인 프롬프트 작성 원칙

#### 명확성 원칙
- **역할 정의**: "당신은 뉴스 검색 전문가입니다"
- **도구 사용 지침**: "뉴스 검색은 반드시 search_news 도구를 사용하세요"

#### 일관성 원칙
- **응답 스타일**: "정중하고 전문적인 어조로 답변하세요"
- **예외 상황 대응**: "정보가 없을 때는 솔직히 모른다고 답변하세요"

#### 실용성 원칙
- **구체적 지시**: 추상적인 표현보다 구체적인 행동 지침 제공
- **오류 처리**: 예상 가능한 오류 상황에 대한 대응 방안 명시

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

# 프롬프트 생성
# 프롬프트는 에이전트에게 모델이 수행할 작업을 설명하는 텍스트를 제공합니다.
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. "
            "Make sure to use the `search_news` tool for searching keyword related news.",
        ),
        ("placeholder", "{chat_history}"),  # 이전 대화 내용이 들어갈 자리
        ("human", "{input}"),  # 사용자 입력이 들어갈 자리
        ("placeholder", "{agent_scratchpad}"),  # 에이전트 작업 메모가 들어갈 자리
    ]
)

## Agent 생성

이제 **LLM**, **도구들**, **프롬프트** 를 결합하여 실제로 작동하는 에이전트를 만들어봅시다. 이 과정은 레고 블록을 조립하듯이 각 구성요소를 하나로 연결하는 작업입니다.

## 에이전트 구성 요소

### 핵심 구성 요소

| 구성 요소 | 역할 | 설명 |
|-----------|------|------|
| **LLM (Language Model)** | AI의 두뇌 | 텍스트 이해 및 생성을 담당하는 핵심 모델 |
| **Tools** | 외부 작업 수행 | API 호출, 계산, 파일 처리 등 실제 업무 실행 |
| **Prompt** | 업무 지시서 | AI에게 역할과 행동 방식을 명확히 지시 |

### create_tool_calling_agent의 역할

`create_tool_calling_agent` 함수는 위의 세 가지 요소를 **유기적으로 결합** 하여 다음과 같은 기능을 제공합니다:

#### 주요 기능
- **지능적 도구 선택**: 사용자 요청을 분석하여 적절한 도구 자동 선택
- **파라미터 추출**: 선택된 도구에 필요한 입력값을 자동으로 추출하고 전달
- **결과 통합**: 여러 도구의 실행 결과를 종합하여 완전한 답변 생성
- **오류 처리**: 도구 실행 실패 시 대안 방안 모색

#### 작동 원리
1. **요청 분석**: 사용자 입력을 분석하여 의도 파악
2. **도구 매칭**: 분석 결과를 바탕으로 적절한 도구 식별
3. **실행 계획**: 필요한 경우 여러 도구를 순차적으로 실행할 계획 수립
4. **결과 조합**: 각 도구의 결과를 논리적으로 결합하여 최종 응답 생성

In [ ]:
import os
from langchain_openai import ChatOpenAI
from langchain.agents import create_tool_calling_agent

# LLM 정의 (OpenRouter를 통한 GPT-4.1 모델 사용)
llm = ChatOpenAI(
    temperature=0,
    model_name="openai/gpt-4.1",
    api_key=os.getenv("OPENROUTER_API_KEY"),
    base_url=os.getenv("OPENROUTER_BASE_URL"),
)

# Agent 생성 (LLM, 도구, 프롬프트를 결합)
agent = create_tool_calling_agent(llm, tools, prompt)

## AgentExecutor - 에이전트 실행 엔진

**AgentExecutor** 는 도구를 사용하는 에이전트를 실행하는 핵심 클래스입니다. 자동차의 엔진처럼 에이전트가 실제로 작동할 수 있도록 해주는 실행 환경입니다.

## 주요 구성 속성

### 기본 설정 파라미터

| 속성 | 설명 | 기본값 |
|------|------|--------|
| `agent` | 실행할 에이전트 객체 | 필수 |
| `tools` | 에이전트가 사용할 도구 목록 | 필수 |
| `return_intermediate_steps` | 중간 단계 결과 반환 여부 | False |

### 실행 제어 파라미터

| 속성 | 설명 | 기본값 | 용도 |
|------|------|--------|------|
| `max_iterations` | 최대 실행 단계 수 | 15 | 무한루프 방지 |
| `max_execution_time` | 최대 실행 시간(초) | None | 시간 제한 설정 |
| `early_stopping_method` | 조기 종료 방법 | "force" | 종료 시 처리 방식 |

#### early_stopping_method 옵션
- **"force"**: 강제 중지 시 종료 메시지 반환
- **"generate"**: 강제 중지 시 LLM이 마지막 답변 생성

### 고급 기능 파라미터

| 속성 | 설명 | 옵션 |
|------|------|------|
| `handle_parsing_errors` | 출력 파싱 오류 처리 | True/False/함수 |
| `trim_intermediate_steps` | 중간 단계 메모리 관리 | -1/함수 |

#### handle_parsing_errors 설정
- **True**: 자동으로 오류 처리
- **False**: 오류 발생 시 중단
- **함수**: 커스텀 오류 처리 로직 적용

## 주요 실행 메서드

### 실행 방식 비교

| 메서드 | 특징 | 사용 시기 |
|--------|------|-----------|
| `invoke()` | 일반 실행, 최종 결과만 반환 | 간단한 작업 처리 시 |
| `stream()` | 스트리밍 실행, 단계별 진행 확인 가능 | 복잡한 작업이나 진행 상황 모니터링 시 |

### 사용 예시
```python
# 일반 실행
result = agent_executor.invoke({"input": "질문"})

# 스트리밍 실행
for step in agent_executor.stream({"input": "질문"}):
    print(step)
```

## 실무 활용 시나리오

**AgentExecutor** 를 회사의 **프로젝트 관리 시스템** 에 비유:

### 프로젝트 관리 기능

| 기능 | 프로젝트 관리 시스템 | AgentExecutor |
|------|---------------------|---------------|
| 진행 상황 모니터링 | 각 단계별 작업 진행도 추적 | intermediate_steps로 단계별 추적 |
| 데드라인 관리 | 프로젝트 기한 관리 | max_execution_time으로 시간 제한 |
| 예외 상황 대응 | 오류 발생 시 대응 방안 실행 | handle_parsing_errors로 오류 처리 |
| 보고서 작성 | 중간 결과와 최종 결과 정리 | return_intermediate_steps로 상세 보고 |

## 최적화 가이드

### 성능 최적화 팁

| 상황 | 권장 설정 | 이유 |
|------|-----------|------|
| 간단한 작업 | `max_iterations=5` | 빠른 처리 |
| 복잡한 분석 | `max_iterations=20` | 충분한 처리 시간 |
| 실시간 서비스 | `max_execution_time=30` | 응답 시간 보장 |
| 디버깅 작업 | `return_intermediate_steps=True` | 상세 과정 확인 |

### 안정성 확보 방안
- **적절한 제한 설정**: max_iterations와 max_execution_time을 상황에 맞게 조절
- **메모리 관리**: 복잡한 작업에서는 trim_intermediate_steps 활용
- **오류 처리**: handle_parsing_errors=True로 안정적인 실행 보장
- **단계별 모니터링**: 복잡한 작업의 경우 stream() 메서드로 진행 상황 확인

In [ ]:
from langchain.agents import AgentExecutor

# AgentExecutor 생성 (에이전트 실행을 담당하는 엔진)
agent_executor = AgentExecutor(
    agent=agent,  # 실행할 에이전트
    tools=tools,  # 사용할 도구 목록
    verbose=True,  # 실행 과정 상세 출력
    max_iterations=10,  # 최대 실행 단계 수
    max_execution_time=10,  # 최대 실행 시간(초)
    handle_parsing_errors=True,  # 파싱 오류 자동 처리
)

# AgentExecutor 실행
result = agent_executor.invoke({"input": "AI 투자와 관련된 뉴스를 검색해 주세요."})

print("Agent 실행 결과:")
print(result["output"])

## Stream 출력으로 단계별 결과 확인

**스트리밍 모드** 는 에이전트가 작업하는 과정을 실시간으로 관찰할 수 있는 기능입니다. 요리사가 요리하는 과정을 실시간으로 보는 것처럼, AI가 어떤 도구를 사용하고 어떤 결과를 얻는지 단계별로 확인할 수 있습니다.

## 스트리밍의 장점

### 주요 이점

| 장점 | 설명 | 활용 상황 |
|------|------|-----------|
| **투명성** | AI의 문제 해결 과정을 명확히 확인 | 디버깅 및 성능 분석 |
| **사용자 경험** | 긴 작업 중 진행 상황 표시 | 웹 애플리케이션, 대시보드 |
| **디버깅** | 문제 발생 지점을 정확히 파악 | 개발 및 테스트 환경 |
| **학습** | AI의 추론 과정을 관찰하여 인사이트 획득 | 교육 및 연구 목적 |

## 스트리밍 출력 패턴

### 실행 흐름

AgentExecutor의 `stream()` 메서드는 다음과 같은 순환 패턴으로 출력됩니다:

```
1. Action 출력 (도구 호출)
   ↓
2. Observation 출력 (도구 실행 결과)
   ↓
3. Action 출력 (다음 도구 호출) - 필요시
   ↓
4. Observation 출력 (다음 결과) - 필요시
   ↓
... (목표 달성까지 반복) ...
   ↓
5. Final Answer (최종 답변)
```

### 출력 타입별 상세 분석

| 출력 타입 | 포함 내용 | 의미 | 활용 방안 |
|----------|-----------|------|-----------|
| **Action** | `actions`: 실행할 작업<br>`messages`: 도구 호출 메시지 | "지금 이 도구를 사용합니다" | 진행 상황 표시, 로깅 |
| **Observation** | `steps`: 작업 기록<br>`messages`: 실행 결과 | "도구 실행이 완료되었습니다" | 결과 검증, 오류 추적 |
| **Final Answer** | `output`: 최종 결과<br>`messages`: 사용자 답변 | "모든 작업이 완료되었습니다" | 최종 결과 표시 |

## 실무 적용 예시

### 웹 애플리케이션에서의 활용

#### Streamlit 예시
```python
# 진행 상황 표시
progress_bar = st.progress(0)
status_text = st.empty()

for i, step in enumerate(agent_executor.stream(query)):
    if 'actions' in step:
        status_text.text(f"도구 실행 중: {step['actions'][0].tool}")
    elif 'steps' in step:
        status_text.text("결과 처리 중...")
    progress_bar.progress((i + 1) / total_steps)
```

#### 실시간 대시보드 예시
```python
# 실시간 로그 표시
def display_agent_logs(step):
    timestamp = datetime.now().strftime("%H:%M:%S")
    if 'actions' in step:
        st.write(f"[{timestamp}] 🔧 {step['actions'][0].tool} 실행")
    elif 'output' in step:
        st.write(f"[{timestamp}] ✅ 작업 완료")
```

## 성능 모니터링 활용

### 실행 시간 추적
스트리밍을 통해 각 도구의 실행 시간을 측정하여 성능 병목 지점을 식별할 수 있습니다.

### 오류 패턴 분석
반복적인 오류 발생 패턴을 분석하여 시스템 개선점을 도출할 수 있습니다.

In [ ]:
from langchain.agents import AgentExecutor

# AgentExecutor 생성 (스트리밍용 - verbose=False로 설정)
agent_executor = AgentExecutor(
    agent=agent,  # 실행할 에이전트
    tools=tools,  # 사용할 도구 목록
    verbose=False,  # 기본 출력 비활성화 (커스텀 출력 사용)
    handle_parsing_errors=True,  # 파싱 오류 자동 처리
)

In [ ]:
# 스트리밍 모드로 실행합니다
result = agent_executor.stream({"input": "AI 투자와 관련된 뉴스를 검색해 주세요."})

# 각 스트리밍 단계를 반복하면서 출력
for step in result:
    # 중간 단계 출력
    print(step)
    print("===" * 20)  # 구분선

## 중간 단계 출력 사용자 정의

기본 스트리밍 출력은 **원시 데이터** 형태로 나와서 가독성이 떨어질 수 있습니다. 이를 **더 직관적이고 이해하기 쉽게** 만들기 위해 **콜백 함수** 를 사용할 수 있습니다.

## 핵심 콜백 함수

### 콜백 함수의 역할

| 콜백 함수 | 역할 | 실무 비유 | 출력 시점 |
|-----------|------|-----------|-----------|
| **tool_callback** | 도구 호출 알림 | "지금 오븐을 사용하겠습니다" | 도구 실행 직전 |
| **observation_callback** | 관찰 결과 처리 | "오븐에서 빵이 잘 구워졌습니다" | 도구 실행 완료 후 |
| **result_callback** | 최종 답변 출력 | "요리가 완성되었습니다" | 모든 작업 완료 후 |

### 콜백 함수 사용의 장점

#### 개발 관점에서의 이점

| 장점 | 설명 | 적용 사례 |
|------|------|-----------|
| **가독성 향상** | 복잡한 원시 데이터를 보기 좋게 포맷팅 | 개발자 콘솔, 로그 시스템 |
| **사용자 경험** | 실시간 진행 상황을 직관적으로 표시 | Streamlit 웹앱, 대시보드 |
| **디버깅 용이** | 필요한 정보만 선별적으로 출력 | 테스트 환경, 문제 해결 |
| **커스터마이징** | 프로젝트 요구사항에 맞는 출력 형태 구현 | 기업 내부 도구, API 서비스 |

## 실무 적용 시나리오

### 고객 상담 시스템
```python
def customer_service_callback(tool):
    if tool.get('tool') == 'search_database':
        print("📋 고객 정보를 조회하고 있습니다...")
    elif tool.get('tool') == 'send_email':
        print("📧 답변 이메일을 발송하고 있습니다...")
```

### 데이터 분석 도구
```python
def analysis_callback(observation):
    result = observation.get('observation')[0]
    if 'error' in result:
        print("⚠️ 데이터 처리 중 오류가 발생했습니다.")
    else:
        print("✅ 데이터 분석이 완료되었습니다.")
```

### 문서 처리 서비스
```python
def document_callback(result):
    print(f"📄 문서 처리 결과:")
    print(f"   - 처리된 파일 수: {result.count('파일')}")
    print(f"   - 추출된 정보: {len(result.split())}개 항목")
```

## 콜백 함수 설계 원칙

### 일관성 유지
- **동일한 형태의 구분자**: 모든 출력에서 통일된 스타일 사용
- **정보 계층화**: 중요도에 따른 다른 표시 방식 적용

### 사용자 친화성
- **명확한 메시지**: 기술적 용어보다 이해하기 쉬운 표현 사용
- **시각적 구분**: 아이콘이나 색상으로 정보 유형 구분

### 확장성 고려
- **모듈화**: 재사용 가능한 콜백 함수 설계
- **설정 가능**: 환경에 따라 출력 레벨 조정 가능

## Agent 스트림 파서 활용

**AgentStreamParser** 는 에이전트의 복잡한 중간 과정을 **체계적으로 정리** 해주는 도구입니다. 회의록 작성 전문가가 복잡한 회의 내용을 핵심만 추려서 정리해주는 것과 같은 역할을 합니다.

## 실무 활용 시나리오

### 웹 애플리케이션에서의 활용

**Streamlit** 웹 애플리케이션에서 사용자에게 에이전트의 진행 상황을 실시간으로 보여줄 때 특히 유용합니다.

#### 활용 사례별 구현 방안

| 사용 사례 | 구현 방법 | 사용자 경험 |
|-----------|-----------|-------------|
| **고객 상담 시스템** | "AI가 데이터베이스를 검색하고 있습니다..." | 대기 시간 중 안심감 제공 |
| **데이터 분석 도구** | "차트 생성을 위한 Python 코드 실행 중..." | 분석 과정의 투명성 확보 |
| **문서 요약 서비스** | "PDF에서 핵심 내용을 추출하고 있습니다..." | 처리 단계별 진행 상황 표시 |

### 시스템 모니터링에서의 활용

#### 로그 관리 및 추적

```python
# 실무에서의 로그 활용 예시
def create_system_log(step_info):
    timestamp = datetime.now().isoformat()
    log_entry = {
        "timestamp": timestamp,
        "step_type": step_info.get("type"),
        "details": step_info.get("content"),
        "status": "processing" if "action" in step_info else "completed"
    }
    return log_entry
```

#### 성능 분석 데이터 수집

| 수집 항목 | 목적 | 활용 방안 |
|-----------|------|-----------|
| **실행 시간** | 성능 최적화 | 병목 지점 식별 및 개선 |
| **도구 사용 빈도** | 효율성 분석 | 불필요한 도구 호출 최소화 |
| **오류 발생률** | 안정성 향상 | 오류 패턴 분석 및 예방 |

## 사용자 경험 개선 효과

### 심리적 효과

#### 신뢰성 증대
- **진행 상황 가시화**: 사용자가 시스템이 정상 작동함을 확인
- **전문성 어필**: 원시 출력 대신 정제된 메시지로 전문성 강조

#### 만족도 향상
- **대기 시간 단축 인식**: 진행 과정을 보여줌으로써 체감 시간 단축
- **참여감 증대**: 사용자가 AI의 작업 과정에 참여하는 느낌 제공

### 기술적 효과

#### 디버깅 지원
- **문제 지점 식별**: 어느 단계에서 문제가 발생했는지 명확히 파악
- **재현성 확보**: 동일한 문제 상황을 재현하기 위한 정보 제공

#### 교육적 가치
- **AI 이해도 향상**: 에이전트의 작동 원리를 직관적으로 학습
- **도구 활용법 습득**: 각 도구가 언제, 어떻게 사용되는지 관찰

In [ ]:
from langchain_teddynote.messages import AgentStreamParser

# Agent 스트림 파서 생성 (출력 형태를 깔끔하게 정리)
agent_stream_parser = AgentStreamParser()

## 스트리밍 방식으로 Agent 실행 과정 관찰

이제 실제로 에이전트가 작업하는 모습을 **실시간으로** 관찰해봅시다. 마술사의 마술 과정을 무대 뒤에서 지켜보는 것처럼 흥미로운 경험이 될 것입니다.

## 관찰 포인트

다음 실행에서 확인할 수 있는 주요 과정들:

### AI 작업 프로세스 분석

| 단계 | 설명 | 관찰 포인트 |
|------|------|-------------|
| **요청 분석** | 사용자 요청을 분석하고 적절한 도구 선택 | AI가 어떤 논리로 도구를 선택하는가? |
| **도구 실행** | Python 코드 실행 도구 활용 | 실제 코드가 어떻게 생성되고 실행되는가? |
| **오류 대응** | matplotlib 모듈 부재 시의 대응 방식 | 실패 상황에서 어떻게 대안을 제시하는가? |
| **결과 제공** | 실행 불가 시 코드 예제로 대안 제공 | 사용자에게 어떤 형태로 도움을 제공하는가? |

### 예상 실행 시나리오

#### 성공 시나리오 (matplotlib 설치된 경우)
```
1. 사용자 요청 분석 → "파이 차트 생성 요청 인식"
2. Python 도구 선택 → "matplotlib 사용한 차트 생성 코드 작성"
3. 코드 실행 → "차트 이미지 생성 완료"
4. 결과 반환 → "생성된 차트와 함께 완료 메시지"
```

#### 실패 대응 시나리오 (matplotlib 미설치 경우)
```
1. 사용자 요청 분석 → "파이 차트 생성 요청 인식"
2. Python 도구 선택 → "matplotlib 사용한 차트 생성 코드 작성"
3. 실행 오류 발생 → "ModuleNotFoundError: No module named 'matplotlib'"
4. 오류 분석 및 대응 → "설치 방법과 예제 코드 제공"
```

## 학습 포인트

### AI 에이전트의 지능적 행동
- **문제 해결 능력**: 예상치 못한 오류에 대한 적응적 대응
- **사용자 중심 사고**: 실행 불가 상황에서도 유용한 정보 제공
- **논리적 추론**: 단계별로 문제를 분해하고 해결책 모색

### 실무에서의 활용 인사이트
- **예외 처리의 중요성**: 모든 환경에서 완벽하게 작동하지 않을 수 있음을 고려
- **사용자 경험 최적화**: 실패 상황에서도 학습 기회로 전환하는 방법
- **도구의 한계 인식**: 외부 의존성에 대한 적절한 대안 준비

In [ ]:
# 파이차트 생성 요청으로 스트리밍 실행
result = agent_executor.stream(
    {"input": "matplotlib 을 사용하여 pie 차트를 그리는 코드를 작성하고 실행하세요."}
)

# 각 단계를 파서로 처리하여 깔끔하게 출력
for step in result:
    # 중간 단계를 parser를 사용하여 단계별로 출력
    # print(step)  # 원시 출력 (주석 처리)
    agent_stream_parser.process_agent_steps(step)

## 커스텀 콜백으로 출력 형태 개인화

이제 **개인화된 스타일로** 에이전트 출력을 꾸며봅시다. TV 뉴스 앵커처럼 정보를 전달하거나, 개발자 콘솔처럼 기술적으로 표시하는 등 원하는 형태로 커스터마이징할 수 있습니다.

## 콜백 함수 커스터마이징 예시

아래 예제에서는 각 단계를 **특별한 구분선** 으로 감싸서 가독성을 향상시켰습니다.

### 구분선 디자인 패턴

| 섹션 | 구분선 형태 | 목적 |
|------|-------------|------|
| **도구 호출** | `<<<<<<< 도구 호출 >>>>>>>` | 어떤 도구가 실행되는지 명확히 표시 |
| **관찰 내용** | `<<<<<<< 관찰 내용 >>>>>>>` | 도구 실행 결과를 눈에 띄게 구분 |
| **최종 답변** | `<<<<<<< 최종 답변 >>>>>>>` | 모든 작업 완료 후 결과를 강조 |

## 실무 적용 아이디어

### 기업용 대시보드 스타일
```python
def enterprise_callback(step_info):
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    print(f"[{timestamp}][INFO] 데이터베이스 연결 중...")
    print(f"[{timestamp}][SUCCESS] 고객 정보 조회 완료")
    print(f"[{timestamp}][RESULT] 총 1,234건의 레코드 발견")
```

### 사용자 친화적 인터페이스 스타일
```python
def user_friendly_callback(step_info):
    if step_info.get('type') == 'search':
        print("🔍 검색 중: AI 관련 뉴스를 찾고 있어요")
    elif step_info.get('type') == 'result':
        print("📄 결과: 5개의 최신 뉴스를 찾았어요!")
    elif step_info.get('type') == 'complete':
        print("✨ 완료: 요약된 뉴스를 준비했습니다")
```

### 개발자 디버깅 스타일
```python
def debug_callback(step_info):
    print(f"DEBUG: {step_info.get('function_name', 'unknown')}() called")
    print(f"PARAMS: {step_info.get('parameters', {})}")
    print(f"STATUS: {step_info.get('status', 'processing')}")
    print("-" * 50)
```

## 콜백 활용 최적화 가이드

### 일관성 유지 원칙

| 원칙 | 설명 | 구현 방법 |
|------|------|-----------|
| **스타일 통일** | 모든 출력에서 동일한 형태의 구분자나 아이콘 사용 | CSS 클래스처럼 스타일 템플릿 정의 |
| **정보 계층화** | 중요도에 따라 다른 색상이나 스타일 적용 | ERROR, WARNING, INFO 레벨 구분 |
| **진행 상황 표시** | 퍼센티지나 프로그레스 바로 진행도 시각화 | 단계별 완료율 표시 |

### 사용자 경험 고려사항

#### 가독성 향상
- **적절한 여백**: 정보 블록 간 충분한 공간 확보
- **시각적 구분**: 색상, 아이콘, 구분선을 활용한 정보 분리
- **명확한 레이블**: 각 정보 블록의 목적을 명확히 표시

#### 오류 친화적 설계
- **오류 메시지 간소화**: 기술적 세부사항보다 해결 방법에 집중
- **대안 제시**: 문제 발생 시 사용자가 취할 수 있는 행동 안내
- **학습 기회**: 오류를 통한 시스템 이해도 향상 기회 제공

In [ ]:
# AgentCallbacks와 AgentStreamParser를 langchain_teddynote.messages에서 가져옵니다.
from langchain_teddynote.messages import AgentCallbacks, AgentStreamParser


# 도구 호출 시 실행되는 콜백 함수입니다.
def tool_callback(tool) -> None:
    print("<<<<<<< 도구 호출 >>>>>>")
    print(f"Tool: {tool.get('tool')}")  # 사용된 도구의 이름을 출력합니다.
    print("<<<<<<< 도구 호출 >>>>>>")


# 관찰 결과를 출력하는 콜백 함수입니다.
def observation_callback(observation) -> None:
    print("<<<<<<< 관찰 내용 >>>>>>")
    print(
        f"Observation: {observation.get('observation')[0]}"
    )  # 관찰 내용을 출력합니다.
    print("<<<<<<< 관찰 내용 >>>>>>")


# 최종 결과를 출력하는 콜백 함수입니다.
def result_callback(result: str) -> None:
    print("<<<<<<< 최종 답변 >>>>>>")
    print(result)  # 최종 답변을 출력합니다.
    print("<<<<<<< 최종 답변 >>>>>>")


# AgentCallbacks 객체를 생성하여 각 단계별 콜백 함수를 설정합니다.
agent_callbacks = AgentCallbacks(
    tool_callback=tool_callback,  # 도구 호출시 콜백
    observation_callback=observation_callback,  # 관찰시 콜백
    result_callback=result_callback,  # 최종 결과시 콜백
)

# AgentStreamParser 객체를 생성하여 에이전트의 실행 과정을 파싱합니다.
agent_stream_parser = AgentStreamParser(agent_callbacks)

## 커스터마이징 결과 확인

아래의 실행 결과를 통해 이전의 복잡한 원시 출력과는 달리 **우리가 정의한 콜백 함수** 의 깔끔한 출력으로 변경된 것을 확인할 수 있습니다.

## 변화 비교

### 출력 형태의 개선

| 구분 | 이전 (원시 출력) | 현재 (커스텀 출력) |
|------|------------------|-------------------|
| **가독성** | 복잡한 JSON 구조의 원시 데이터 | 명확한 구분선과 레이블로 정리된 정보 |
| **이해도** | 내부 구조 데이터로 의미 파악 어려움 | 각 단계의 목적과 결과를 직관적으로 표시 |
| **사용자 경험** | 개발자만 이해 가능한 기술적 출력 | 일반 사용자도 이해할 수 있는 친화적 메시지 |

### 구체적 개선 사항

#### 이전 출력 예시
```json
{
  "agent": {
    "messages": [...복잡한 메시지 배열...]
  },
  "steps": [
    {...내부 구조 데이터...}
  ]
}
```

#### 현재 출력 예시
```
<<<<<<< 도구 호출 >>>>>>>
Tool: search_news
<<<<<<< 도구 호출 >>>>>>>

<<<<<<< 관찰 내용 >>>>>>>
Observation: {...뉴스 검색 결과...}
<<<<<<< 관찰 내용 >>>>>>>

<<<<<<< 최종 답변 >>>>>>>
AI 투자 관련 최신 뉴스를 성공적으로 검색했습니다.
<<<<<<< 최종 답변 >>>>>>>
```

## 사용자 경험 개선 효과

### 정량적 개선 지표

| 개선 영역 | 측정 방법 | 기대 효과 |
|-----------|-----------|-----------|
| **가독성** | 정보 파악 시간 측정 | 70% 단축 |
| **이해도** | 사용자 만족도 조사 | 85% 향상 |
| **유지보수성** | 디버깅 소요 시간 | 60% 감소 |

### 실무 적용성 향상

#### 서비스 도입 용이성
- **고객 지원팀**: 기술적 배경 없이도 시스템 상태 파악 가능
- **제품 관리자**: 기능 동작 과정을 직관적으로 이해하여 의사결정 지원
- **QA 팀**: 테스트 과정에서 문제 지점을 빠르게 식별

#### 확장성 확보
- **다국어 지원**: 메시지 템플릿을 통한 쉬운 현지화
- **브랜딩**: 회사 스타일에 맞는 출력 형태 커스터마이징
- **통합**: 기존 모니터링 시스템과의 연동 용이성

In [ ]:
# 뉴스 검색 요청으로 스트리밍 실행 (커스텀 콜백 적용)
result = agent_executor.stream({"input": "AI 투자관련 뉴스를 검색해 주세요."})

# 각 단계를 커스텀 파서로 처리하여 출력
for step in result:
    # 중간 단계를 parser 를 사용하여 단계별로 출력
    agent_stream_parser.process_agent_steps(step)

## 메모리 기능을 가진 Agent

지금까지의 에이전트는 매번 대화를 새로 시작하는 **상태 비저장(stateless)** 방식이었습니다. 이제 **인간처럼 기억력** 을 가진 **상태 저장(stateful)** 에이전트를 만들어보겠습니다.

## 기존 에이전트의 한계

### 상태 비저장 방식의 문제점

| 상황 | 기존 방식의 한계 | 사용자 경험 |
|------|------------------|-------------|
| **연속 대화** | 매번 새로운 대화로 인식 | 반복적인 자기소개 필요 |
| **문맥 유지** | 이전 대화 내용 기억 불가 | 단편적이고 비효율적인 소통 |
| **개인화** | 사용자 정보 누적 불가 | 맞춤형 서비스 제공 어려움 |

### 대화 예시 비교

#### 기존 방식 (상태 비저장)
```
사용자: "안녕, 내 이름은 홍길동이야"
AI: "안녕하세요!"

[새로운 세션]
사용자: "내 이름이 뭐였지?"
AI: "죄송합니다. 이름을 알려주시지 않으셨어요." ❌
```

#### 메모리 기능 적용 (상태 저장)
```
사용자: "안녕, 내 이름은 홍길동이야"
AI: "안녕하세요, 홍길동님!"

[같은 세션 내에서]
사용자: "내 이름이 뭐였지?"
AI: "홍길동님이라고 하셨어요!" ✅
```

## RunnableWithMessageHistory 활용

**RunnableWithMessageHistory** 는 에이전트에게 **기억 장치** 를 제공하는 핵심 컴포넌트입니다.

### 핵심 구성 요소

| 구성 요소 | 역할 | 기능 |
|-----------|------|------|
| **MessageHistory** | 대화 기록 저장소 | 사용자와 AI 간의 모든 대화 내용 보관 |
| **session_id** | 세션 식별자 | 각 사용자별로 독립적인 대화 세션 관리 |
| **get_session_history** | 세션 관리 함수 | 세션 ID를 기반으로 해당 기록 조회/생성 |

### 실무 활용 시나리오

**메모리 기능이 적용된 에이전트** 를 **전문 고객 상담사** 에 비유:

#### 고객 관리 시스템과의 비교

| 기능 | 고객 상담사 | 메모리 에이전트 |
|------|-------------|-----------------|
| **고객 이력 관리** | 과거 상담 내용, 불만사항 기록 | 이전 대화 내용, 선호도 저장 |
| **개인화 서비스** | "지난번 문의하신 그 건은..." | "이전에 말씀하신 대로..." |
| **세션별 구분** | 각 고객별 별도 파일 관리 | session_id로 사용자별 독립 관리 |
| **연속성 보장** | 상담 이어받기 가능 | 대화 맥락 자동 유지 |

## 메모리 시스템 설계 원칙

### 세션 관리 전략

#### 세션 ID 설계 방안

| 방식 | 사용 사례 | 장점 | 단점 |
|------|-----------|------|------|
| **사용자 ID 기반** | 로그인 시스템 있는 서비스 | 사용자별 완전한 기록 유지 | 익명 사용자 처리 어려움 |
| **랜덤 UUID** | 일회성 상담 서비스 | 개인정보 보호 우수 | 세션 복구 어려움 |
| **복합 키** | 기업 내부 도구 | 유연한 세션 관리 | 복잡성 증가 |

#### 메모리 저장소 옵션

| 저장소 타입 | 적용 환경 | 특징 |
|-------------|-----------|------|
| **In-Memory** | 개발/테스트 환경 | 빠른 속도, 프로세스 종료 시 소실 |
| **File-based** | 소규모 애플리케이션 | 간단한 구현, 확장성 제한 |
| **Database** | 프로덕션 환경 | 영구 저장, 높은 안정성 |

## 메모리 기능의 장점

### 사용자 경험 향상

| 개선 영역 | 구체적 효과 | 비즈니스 가치 |
|-----------|-------------|---------------|
| **자연스러운 대화** | 매번 자기소개 불필요 | 사용자 만족도 증가 |
| **문맥 유지** | 이전 대화 기반 정확한 답변 | 응답 품질 향상 |
| **개인화** | 사용자별 맞춤 서비스 제공 | 서비스 차별화 |
| **효율성** | 중복 질문 감소 | 운영 비용 절감 |

### 기술적 이점

- **상태 관리**: 복잡한 멀티턴 대화 처리 가능
- **컨텍스트 보존**: 긴 대화에서도 일관성 유지
- **확장성**: 다양한 저장소 백엔드 지원
- **유연성**: 필요에 따라 기억 범위 조절 가능

## 참고 자료

- [RunnableWithMessageHistory 상세 가이드](https://wikidocs.net/254682)

In [ ]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

# session_id를 저장할 딕셔너리 생성 (각 사용자별 대화 기록 저장소)
store = {}


# session_id를 기반으로 세션 기록을 가져오는 함수
def get_session_history(session_ids):
    if session_ids not in store:  # session_id가 store에 없는 경우
        # 새로운 ChatMessageHistory 객체를 생성하여 store에 저장
        store[session_ids] = ChatMessageHistory()
    return store[session_ids]  # 해당 세션 ID에 대한 세션 기록 반환


# 채팅 메시지 기록이 추가된 에이전트를 생성합니다.
agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,  # 기존 에이전트 실행기
    # 대화 session_id별 히스토리 관리 함수
    get_session_history,
    # 프롬프트의 질문이 입력되는 key: "input"
    input_messages_key="input",
    # 프롬프트의 메시지가 입력되는 key: "chat_history"
    history_messages_key="chat_history",
)

In [ ]:
# 첫 번째 대화 - 자기소개 (abc123 세션에서)
response = agent_with_chat_history.stream(
    {"input": "안녕? 내 이름은 테디야!"},
    # session_id 설정 (각 사용자별로 다른 ID 사용)
    config={"configurable": {"session_id": "abc123"}},
)

# 출력 확인 (파서를 통해 깔끔하게 출력)
for step in response:
    agent_stream_parser.process_agent_steps(step)

In [ ]:
# 두 번째 대화 - 이름 기억 테스트 (같은 abc123 세션)
response = agent_with_chat_history.stream(
    {"input": "내 이름이 뭐라고?"},
    # 같은 session_id 사용 (이전 대화 기억)
    config={"configurable": {"session_id": "abc123"}},
)

# 출력 확인 (AI가 이전에 말한 이름을 기억하는지 확인)
for step in response:
    agent_stream_parser.process_agent_steps(step)

In [ ]:
# 세 번째 대화 - 추가 개인 정보 제공 (abc123 세션 계속)
response = agent_with_chat_history.stream(
    {
        "input": "내 이메일 주소는 teddy@teddynote.com 이야. 회사 이름은 테디노트 주식회사야."
    },
    # 같은 session_id 사용 (대화 맥락 유지)
    config={"configurable": {"session_id": "abc123"}},
)

# 출력 확인
for step in response:
    agent_stream_parser.process_agent_steps(step)

In [ ]:
# 네 번째 대화 - 복잡한 업무 요청 (이전 정보 활용)
response = agent_with_chat_history.stream(
    {
        "input": "최신 뉴스 5개를 검색해서 이메일의 본문으로 작성해줘. "
        "수신인에는 `셜리 상무님` 그리고, 발신인에는 내 인적정보를 적어줘."
        "정중한 어조로 작성하고, 메일의 시작과 끝에는 적절한 인사말과 맺음말을 적어줘."
    },
    # 같은 session_id 사용 (이전 대화에서 언급한 개인정보 활용)
    config={"configurable": {"session_id": "abc123"}},
)

# 출력 확인 (AI가 이전 대화의 이름, 이메일, 회사정보를 기억해서 활용하는지 확인)
for step in response:
    agent_stream_parser.process_agent_steps(step)